In [2]:
import h5py
import scipy as sp
import pandas as pd
import timeit
import matplotlib.pyplot as pl
from time import sleep
from numpy import *
from sklearn.decomposition import PCA

In [3]:
with h5py.File('images_training.h5','r') as H:
    data_train = copy(H['data'])
with h5py.File('labels_training.h5','r') as H:
    label_train = copy(H['label'])
with h5py.File('images_testing.h5 ','r') as H:
    data_test = copy(H['data'])
with h5py.File('labels_testing_2000.h5','r') as H:
    label_test = copy(H['label'])

In [19]:
class optimization:
    '''
    Initialize the  parameters
    C: C controls the margin of SVM
    tol: tolerance set the stopping criterion
    '''
    def __init__(self,matrixIndex, inputClassLabels, C, toler, k_Type):  
        self.X = matrixIndex 
        self.label_matrix = inputClassLabels
        self.tol = toler
        self.C = C
        self.m = shape(matrixIndex)[0]
        self.alpha = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) 
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernel(self.X, self.X[i,:], k_Type)
 # select next j for ahpla which            
def selectJ(i, opt, Ei):        
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    opt.eCache[i] = [1,Ei]  #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(opt.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #traverse ecache values to find the value that maximize delta 
            if k == i: 
                continue 
            Ek = calculateEk(opt, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE 
                Ej = Ek
        return maxK, Ej
    else:   
        j = selectJrandom(i, opt.m)
        Ej = calculateEk(opt, j)
    return j, Ej
    
#select another j between i and m
def selectJrandom(i,m): 
    j=i 
    while (j==i):
        j = int(random.uniform(0,m))
    return j

#clip the alpha with constrainng direction
def set_Alpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

def kernel(X, A, k_Type): 
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if k_Type[0]=='lin':  #linear kernel
        K = dot(X,A.T)   
    elif k_Type[0]=='rbf': #Gaussian kernel
        for j in range(m):
            deltaRow = X[j,:] - A 
            K[j] = deltaRow*deltaRow.T 
        K = exp(K/(-1*k_Type[1]**2))  
    else: 
        raise NameError('Kernel not recognized')
    return K

#After chaning any alpha, update the new value of E_k in the ecache
def resetEk(opt, k): 
    Ek = calculateEk(opt, k)
    opt.eCache[k] = [1,Ek]

#Ek = f(x_k)-y_k
def calculateEk(opt, k): #calculate the E
        fXk = float(multiply(opt.alpha,opt.label_matrix).T*opt.K[:,k] + opt.b) # f(x_k) = a*y*x+b
        Ek = fXk - float(opt.label_matrix[k])
        return Ek     

# the inner loop keep optimizing the alpha pairs     
def smoInnerLoop(i, opt):
    Ei = calculateEk(opt, i)
    # if alpha do not satisfy the KKT condition, then optimize it with alpha_j
    if ((opt.label_matrix[i]*Ei < -opt.tol) and (opt.alpha[i] < opt.C)) or ((opt.label_matrix[i]*Ei > opt.tol) and (opt.alpha[i] > 0)):
        j,Ej = selectJ(i, opt, Ei) 
        alphaIold = opt.alpha[i].copy()
        alphaJold = opt.alpha[j].copy()
        if (opt.label_matrix[i] != opt.label_matrix[j]): #use L ,H to clip the alpha
            L = max(0, opt.alpha[j] - opt.alpha[i])
            H = min(opt.C, opt.C + opt.alpha[j] - opt.alpha[i])
        else:
            L = max(0, opt.alpha[j] + opt.alpha[i] - opt.C)
            H = min(opt.C, opt.alpha[j] + opt.alpha[i])
        if L==H: 
            return 0
        eta = 2.0 * opt.K[i,j] - opt.K[i,i] - opt.K[j,j] #changed the parameter
        if eta >= 0: 
            return 0
        opt.alpha[j] -= opt.label_matrix[j]*(Ei - Ej)/eta
        opt.alpha[j] = set_Alpha(opt.alpha[j],H,L)
        resetEk(opt, j) 
        if (abs(opt.alpha[j] - alphaJold) < 0.00001): 
            return 0
        opt.alpha[i] += opt.label_matrix[j]*opt.label_matrix[i]*(alphaJold - opt.alpha[j])#update alpha
        resetEk(opt, i)                
        b1 = opt.b - Ei- opt.label_matrix[i]*(opt.alpha[i]-alphaIold)*opt.K[i,i] - opt.label_matrix[j]*(opt.alpha[j]-alphaJold)*opt.K[i,j]
        b2 = opt.b - Ej- opt.label_matrix[i]*(opt.alpha[i]-alphaIold)*opt.K[i,j]- opt.label_matrix[j]*(opt.alpha[j]-alphaJold)*opt.K[j,j]
        if (0 < opt.alpha[i]) and (opt.C > opt.alpha[i]): 
            opt.b = b1
        elif (0 < opt.alpha[j]) and (opt.C > opt.alpha[j]): 
            opt.b = b2
        else: 
            opt.b = (b1 + b2)/2.0
        return 1
    else: 
        return 0
#SMO outer loop traverse all alpha and make sure all alpha satisfy KKT
def SMO(matrixIndex, inputClassLabels, C, toler, maxIter,k_Type):    
    opt = optimization(mat(matrixIndex),mat(inputClassLabels).transpose(),C,toler, k_Type)
    iteration = 0
    entire = True
    alphaPairsChanged = 0
    while (iteration < maxIter) and ((alphaPairsChanged > 0) or (entire)): 
        alphaPairsChanged = 0
        if entire:   
            for i in range(opt.m):        
                alphaPairsChanged += smoInnerLoop(i,opt)
            iteration += 1
        else:  #traverse all non-bound alpha
            nonBound = nonzero((opt.alpha.A > 0) * (opt.alpha.A < C))[0]
            for i in nonBound:
                alphaPairsChanged += smoInnerLoop(i,opt)
            iteration += 1
        if entire: #change loop if traverse all sample
            entire = False 
        elif (alphaPairsChanged == 0): 
            entire = True  
    return opt.b,opt.alpha

#reshape the trainning data
def loadData(data,label,categ = 1): #categories defalt by 1
    m = shape(data)[0]
    rbfLabel = []
    data_2D = reshape(data,(m,784))
    for i in range(m):
        if label[i] == categ: 
            rbfLabel.append(1)
        else: 
            rbfLabel.append(-1)       
    return data_2D, rbfLabel

#load the data and calculate result
def fit(train_size,test_size,cat=5,k_Type=('rbf', 10),C = 1):
    trainArray,trainLabelArray = loadData(data_train[0:train_size],label_train[0:train_size],categ = cat)  
    testArray, testLabelArray  = loadData(data_test[0:test_size],label_test,categ = cat)  
    b,alpha = SMO(trainArray, trainLabelArray, C, 0.0001, 10000, k_Type)
    data_matrix=mat(trainArray)  
    label_matrix = mat(trainLabelArray).transpose()
    sv_index=nonzero(alpha.A>0)[0]  
    s_vector=data_matrix[sv_index]   
    labelSV = label_matrix[sv_index];
    data_matrix=mat(testArray)
    label_matrix = mat(testLabelArray).transpose()
    m,n = shape(data_matrix)
     #calculate the accuracy for one category
    predict_one_cat = zeros(m)
    # prediction result
    MatchCount = 0
    for i in range(len(predict_one_cat)):
        predict_one_cat[i] = -1
    for i in range(m):
        result =0
        kernelEval = kernel(s_vector,data_matrix[i,:],k_Type)
        result=kernelEval.T * multiply(labelSV,alpha[sv_index]) + b 
        if sign(result)[0,0]>0:
            predict_one_cat[i]=cat
        if sign(result)==sign(testLabelArray[i]): 
            MatchCount += 1
    return predict_one_cat,MatchCount/m
 
#score one category
def score(C,train_size=3000,test_size=2000,cat=1,k =('rbf', 10)):  
    match = fit(train_size = train_size,test_size =test_size,C=C)[1]
    return match
    
#one versus all
def scoreAllCat(C,train_size=500,test_size=100):
    cate = range(10)
    one_cat_acc =0
    for i in cate:
        one_cat_acc = one_cat_acc+fit(cat =i, C=C,train_size=train_size,test_size=test_size)
    return one_cat_acc/9

#predict all category
def predict(C,train_size=3000,test_size=2000,cat=1,k =('rbf', 10)):
    predict = fit(train_size = train_size,test_size =test_size,C=C,cat =0)[0]
    for i in range(1,len(predict)):
        new_predict = fit(train_size = train_size,test_size =test_size,C=C,cat =i)[0]
        for j in range(len(new_predict)):
            if new_predict[j]>0:
                predict[j]=new_predict[j]
    return predict

In [5]:
#Test data on small set:3000 train sample
H = [1.0,5.0,10.0,20.0,25.0,50.0,100.0]
print("RBF kernel: ")
for i in H:
    start_time = timeit.default_timer() 
    print("C: ",i,"accuracy: ", score(i,k =('rbf', 10)))
    print("Total time :", timeit.default_timer() - start_time , 's')

RBF kernel: 
C:  1.0 accuracy:  0.8615
Total time : 253.30631812399952 s
C:  5.0 accuracy:  0.8635
Total time : 179.43335787701653 s
C:  10.0 accuracy:  0.6045
Total time : 335.13926869499846 s
C:  20.0 accuracy:  0.869
Total time : 180.68416856098338 s
C:  25.0 accuracy:  0.8665
Total time : 181.58253930599312 s
C:  50.0 accuracy:  0.827
Total time : 185.83314241899643 s
C:  100.0 accuracy:  0.8535
Total time : 174.83070236400818 s


In [6]:
#Linear kernel with different C  on small set:3000 train sample
H = [1.0,5.0,10.0]
print("linear kernel: ")
for i in H:
    start_time = timeit.default_timer() 
    print("C: ",i,"accuracy: ", score(i,k =('lin', 0)))
    print("Total time :", timeit.default_timer() - start_time , 's')

RBF kernel: 
C:  1.0 accuracy:  0.8615
Total time : 248.97746718401322 s
C:  5.0 accuracy:  0.7855
Total time : 207.43849651701748 s
C:  10.0 accuracy:  0.841
Total time : 178.89588023300166 s


In [100]:
#calculate accuracy for one category
socre(20.0,train_size=30000,test_size=2000,cat=1)

In [21]:
#predict all category
predict(20.0)